<a href="https://colab.research.google.com/github/AdarshaMohitSahu/FromGemini/blob/main/Loan_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [2]:
import re
import csv

def clean_report_to_csv(file_path, output_csv_path):
    """
    Cleans the report text file and converts it to a CSV file.

    Args:
        file_path (str): The path to the input text file.
        output_csv_path (str): The path where the output CSV file will be saved.
    """
    # Read the file content
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    cleaned_data = []
    current_product_info1 = ""
    current_product_info2 = ""

    # Regex patterns
    # Matches lines like "REPORT ID:", "INDIAN BANK", "PAGE NO :", "------"
    header_footer_pattern = re.compile(
        r"^(REPORT ID:|INDIAN BANK|AREA:|BRANCH\s*:|PAGE NO\s*:|PROC DATE:|-{50,}|SL-NO\s+ACCT NO\s+CUSTOMER NAME|^\s*$)" # Added closing parenthesis here
    )
    # Matches product total or GL total lines, e.g., "PRODUCT TOTAL ="
    summary_total_pattern = re.compile(r"^\s*(PRODUCT TOTAL|GL-WISE TOTAL)\s*=")

    # Matches the first product/category line, e.g., "06083INR6341505002 Retail-Housing-AWAS"
    product_info1_pattern = re.compile(r"^\s*(\w{10,}\s+.*)")

    # Matches the second product/category line, e.g., "4505/0050 IndAwas MIG1 FXD RES MCLR" or "2996/0033 Retail TL- ASHIANA HSG FL 9.65"
    product_info2_pattern = re.compile(r"^\s*([A-Za-z0-9\/\-\s]+\d{2,}(\.\d{2})?)$")
    # A more specific pattern if the second line always starts with a code like XXXX/YYYY
    product_info2_specific_pattern = re.compile(r"^\s*(\w{4}/\w{4}\s+.*)")


    # Main data line pattern - this is crucial and might need adjustments
    # It tries to capture all fields based on typical spacing and data types.
    # SL-NO ACCT NO CUSTOMER NAME TERM ACC-OPEN-DT LIMIT INT-RT PROD-RT DIFF DRAWING LIMIT BAL OUTSTANDING IRREGULARITY
    data_pattern = re.compile(
        r"^\s*(?P<SL_NO>\d+)\s+"                                   # SL-NO
        r"(?P<ACCT_NO>\d+)\s+"                                     # ACCT NO
        r"(?P<CUSTOMER_NAME>.+?)\s+"                               # CUSTOMER NAME (non-greedy)
        r"(?P<TERM>\d{2,3})\s+"                                    # TERM (2 or 3 digits)
        r"(?P<ACC_OPEN_DT>\d{2}/\d{2}/\d{4})\s+"                   # ACC-OPEN-DT
        r"(?P<LIMIT>[\d,]+\.\d{2})\s+"                             # LIMIT
        r"(?P<INT_RT>\d{2}\.\d{4})\s+"                             # INT-RT
        r"(?P<PROD_RT>\d{1,2}\.\d{2})\s+"                          # PROD-RT
        r"(?P<DIFF>[-]?\d{1,2}\.\d{2})\s+"                         # DIFF
        r"(?P<DRAWING_LIMIT>[\d,]+\.\d{2})\s+"                     # DRAWING LIMIT
        r"(?P<BAL_OUTSTANDING>[\d,]+\.\d{2})\s+"                   # BAL OUTSTANDING
        r"(?P<IRREGULARITY>[\d,]+\.\d{2})\s*$"                     # IRREGULARITY
    )

    # Alternative pattern for lines where customer name might be shorter or followed by fewer spaces before TERM
    data_pattern_alt = re.compile(
        r"^\s*(?P<SL_NO>\d+)\s+"
        r"(?P<ACCT_NO>\d+)\s+"
        r"(?P<CUSTOMER_NAME>[A-Za-z\s\.]+?)\s{2,}"  # Customer name followed by at least 2 spaces
        r"(?P<TERM>\d{2,3})\s+"
        r"(?P<ACC_OPEN_DT>\d{2}/\d{2}/\d{4})\s+"
        r"(?P<LIMIT>[\d,]+\.\d{2})\s+"
        r"(?P<INT_RT>\d{2}\.\d{4})\s+"
        r"(?P<PROD_RT>\d{1,2}\.\d{2})\s+"
        r"(?P<DIFF>[-]?\d{1,2}\.\d{2})\s+"
        r"(?P<DRAWING_LIMIT>[\d,]+\.\d{2})\s+"
        r"(?P<BAL_OUTSTANDING>[\d,]+\.\d{2})\s+"
        r"(?P<IRREGULARITY>[\d,]+\.\d{2})\s*$"
    )

    # Tiered rate lines
    tiered_rate_marker = re.compile(r"^\s*TIERED RATE\s*:")


    skip_next_lines = 0 # To skip lines after TIERED RATE marker

    for line_num, line_content in enumerate(lines):
        line = line_content.strip()

        if skip_next_lines > 0:
            skip_next_lines -= 1
            continue

        if header_footer_pattern.search(line) or summary_total_pattern.search(line):
            continue

        # Check for TIERED RATE marker
        if tiered_rate_marker.search(line):
            # This indicates a special rate section; we might want to skip the rate lines that follow
            # For this example, we'll skip the next 2-3 lines which usually describe the tiers.
            # You might need to adjust this count or implement more specific logic.
            skip_next_lines = 2 # Adjust as needed, some have 2, some more for rate conditions
            continue


        # Attempt to match product information lines
        match_prod1 = product_info1_pattern.match(line)
        if match_prod1:
            # Check if it's not a data line continuation (e.g., a long customer name that looks like a product line)
            # This is a heuristic: if the line contains digits typical of account data, it might not be a product line.
            if not re.search(r'\d{2}/\d{2}/\d{4}', line) and not re.search(r'\s\d+\.\d{2}\s', line):
                 # Check if it's not a summary line that might have been missed.
                if "TOTAL =" not in line and "PAGE NO" not in line and "SL-NO" not in line :
                    current_product_info1 = match_prod1.group(1).strip()
                    current_product_info2 = "" # Reset second product line
                    # print(f"DEBUG: Matched Prod1: {current_product_info1}")
                    continue # Move to next line after capturing product info


        match_prod2_specific = product_info2_specific_pattern.match(line)
        if match_prod2_specific and current_product_info1: #Only match if prod1 is set
            if "TOTAL =" not in line and "PAGE NO" not in line and "SL-NO" not in line:
                current_product_info2 = match_prod2_specific.group(1).strip()
                # print(f"DEBUG: Matched Prod2 (Specific): {current_product_info2}")
                continue

        match_prod2 = product_info2_pattern.match(line)
        if match_prod2 and current_product_info1 and not current_product_info2: # Only match if prod1 is set and prod2 is not
             if "TOTAL =" not in line and "PAGE NO" not in line and "SL-NO" not in line:
                current_product_info2 = match_prod2.group(1).strip()
                # print(f"DEBUG: Matched Prod2 (General): {current_product_info2}")
                continue


        # Attempt to match data lines
        match_data = data_pattern.match(line)
        if not match_data:
            match_data = data_pattern_alt.match(line)

        if match_data:
            row = match_data.groupdict()
            # Clean numeric fields by removing commas
            for key in ['LIMIT', 'DRAWING_LIMIT', 'BAL_OUTSTANDING', 'IRREGULARITY']:
                if row[key]:
                    row[key] = row[key].replace(',', '')

            row['PRODUCT_INFO_1'] = current_product_info1
            row['PRODUCT_INFO_2'] = current_product_info2
            cleaned_data.append(row)
            # print(f"DEBUG: Matched Data: {row['SL_NO']}, {row['ACCT_NO']}, {row['CUSTOMER_NAME']}")
            continue # Successfully processed data line

        # If no pattern matched and it's not an empty/header/footer line, it might be an unhandled case
        # or a continuation of a previous field (e.g. very long customer name).
        # For simplicity, this example doesn't handle multi-line concatenation for fields like CUSTOMER_NAME
        # beyond what the regex attempts. More complex logic would be needed here if names are split.
        # Example: if cleaned_data and not line.strip().startswith(tuple(str(i) for i in range(10))):
        # cleaned_data[-1]['CUSTOMER_NAME'] += " " + line.strip() # Append to previous customer name

    # Define CSV headers
    headers = [
        'SL_NO', 'ACCT_NO', 'CUSTOMER_NAME', 'TERM', 'ACC_OPEN_DT',
        'LIMIT', 'INT_RT', 'PROD_RT', 'DIFF', 'DRAWING_LIMIT',
        'BAL_OUTSTANDING', 'IRREGULARITY', 'PRODUCT_INFO_1', 'PRODUCT_INFO_2'
    ]

    # Write to CSV
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for row_data in cleaned_data:
            # Ensure all possible keys are present or defaulted
            full_row = {header: row_data.get(header, "") for header in headers}
            writer.writerow(full_row)

    print(f"Successfully cleaned data and saved to {output_csv_path}")
    print(f"Number of records extracted: {len(cleaned_data)}")

# --- Main execution ---
# In Colab, upload your file first, then set the path.
# For example, if your file is named "06083_20250131_LoansBalanceFile-lond2390.txt"
# and you've uploaded it directly to the Colab session storage:
file_path = '/content/06083_20250331_LoansBalanceFile-lond2390.txt'
output_csv_path = 'BalanceLoans_Mar.csv'

# Before running, ensure the file_path is correct and the file exists in your Colab environment.
# You might need to upload the file to Colab first.
try:
    with open(file_path, 'r') as f_test: # Test if file exists
        pass
    clean_report_to_csv(file_path, output_csv_path)
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please make sure you have uploaded the file to your Colab session and the path is correct.")

Successfully cleaned data and saved to BalanceLoans_Mar.csv
Number of records extracted: 1634


In [1]:
import re
import csv

def clean_monthly_report_to_csv(file_path, output_csv_path):
    """
    Cleans the monthly loans opened/closed report and converts it to a CSV file.

    Args:
        file_path (str): The path to the input text file.
        output_csv_path (str): The path where the output CSV file will be saved.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    cleaned_data = []
    current_report_section = ""  # To store "ACCOUNTS CLOSED" or "ACCOUNTS OPENED"
    current_gl_product_group = "" # e.g., 06083INR6341505021 Retail-Education-India-public
    current_product_code_desc = ""  # e.g., 2996/0094 Retail TL-Edu Reputed Ins

    # Regex patterns
    header_pattern = re.compile(
        r"^(REPORT ID:|INDIAN BANK|AREA:|BRANCH CODE\s*:|PAGE NO\s*\d+|PROC DATE:|MONTHLY REPORT ON LOANS OPENED/CLOSED|-{50,}|SL-NO\s+ACCOUNT-NO\s+CUSTOMER-NO|^\s*$)"
    )
    section_header_pattern = re.compile(r"^\s*(ACCOUNTS (CLOSED|OPENED) DURING THE MONTH)\s*$")
    gl_product_group_pattern = re.compile(r"^\s*(\w{5}INR\d+\s+.*)") # e.g., 06083INR6341505021 Retail-Education-India-public
    product_code_desc_pattern = re.compile(r"^\s*(\w{4}/\w{4}\s+.*)") # e.g., 2996/0094 Retail TL-Edu Reputed Ins
    underscore_separator_pattern = re.compile(r"^_+$")
    end_of_report_pattern = re.compile(r"^\(ASSISTANT MANAGER/MANAGER\)")


    # SL-NO ACCOUNT-NO CUSTOMER-NO OPN-CLS DT CUSTOMER-NAME LIMIT LOAN-BALANCE INT-RATE
    data_pattern = re.compile(
        r"^\s*(?P<SL_NO>\d+)\s+"
        r"(?P<ACCOUNT_NO>[\d-]+)\s+"  # Account number with potential hyphen
        r"(?P<CUSTOMER_NO>[\d-]+)\s+" # Customer number with potential hyphen
        r"(?P<OPN_CLS_DT>\d{2}/\d{2}/\d{4})\s+"
        r"(?P<CUSTOMER_NAME>.+?)\s{2,}"  # Customer name (non-greedy, followed by at least 2 spaces)
        r"(?P<LIMIT>[\d,]+\.\d{2})\s+"
        r"(?P<LOAN_BALANCE>[\d,]+\.\d{2})\s+"
        r"(?P<INT_RATE>\d+\.\d{2})\s*$"
    )
    # Handle cases where customer name might be very short, or int rate is missing (though not expected from format)
    data_pattern_alt_for_missing_int_rate = re.compile(
        r"^\s*(?P<SL_NO>\d+)\s+"
        r"(?P<ACCOUNT_NO>[\d-]+)\s+"
        r"(?P<CUSTOMER_NO>[\d-]+)\s+"
        r"(?P<OPN_CLS_DT>\d{2}/\d{2}/\d{4})\s+"
        r"(?P<CUSTOMER_NAME>.+?)\s{2,}"
        r"(?P<LIMIT>[\d,]+\.\d{2})\s+"
        r"(?P<LOAN_BALANCE>[\d,]+\.\d{2})\s*$" # INT_RATE is optional here
    )


    for line_content in lines:
        line = line_content.strip()

        if header_pattern.search(line) or end_of_report_pattern.search(line) or underscore_separator_pattern.search(line):
            continue

        match_section = section_header_pattern.match(line)
        if match_section:
            current_report_section = match_section.group(1).strip()
            # Skip the "=================================" line that follows
            if "ACCOUNTS CLOSED" in current_report_section or "ACCOUNTS OPENED" in current_report_section:
                 # Reset product info when section changes
                current_gl_product_group = ""
                current_product_code_desc = ""
            continue

        match_gl_group = gl_product_group_pattern.match(line)
        if match_gl_group:
            current_gl_product_group = match_gl_group.group(1).strip()
            current_product_code_desc = "" # Reset sub-product when GL group changes
            continue

        match_prod_code = product_code_desc_pattern.match(line)
        if match_prod_code:
            current_product_code_desc = match_prod_code.group(1).strip()
            continue

        # Data line processing
        match_data = data_pattern.match(line)
        if not match_data : # try alternative if first fails
             match_data = data_pattern_alt_for_missing_int_rate.match(line)


        if match_data:
            row = match_data.groupdict()
            row['STATUS_SECTION'] = current_report_section
            row['GL_PRODUCT_GROUP'] = current_gl_product_group
            row['PRODUCT_CODE_DESC'] = current_product_code_desc

            # Clean numeric fields
            for key in ['LIMIT', 'LOAN_BALANCE']:
                if row[key]:
                    row[key] = row[key].replace(',', '')

            if 'INT_RATE' not in row or row['INT_RATE'] is None: # Handle if alt pattern matched
                row['INT_RATE'] = ""

            cleaned_data.append(row)
            continue

        # If a line is not empty and doesn't match any pattern, print it for debugging
        # This can help identify lines that need new regex rules or adjustments
        # if line:
        # print(f"DEBUG: Unhandled line: {line}")


    headers = [
        'STATUS_SECTION', 'GL_PRODUCT_GROUP', 'PRODUCT_CODE_DESC',
        'SL_NO', 'ACCOUNT_NO', 'CUSTOMER_NO', 'OPN_CLS_DT',
        'CUSTOMER_NAME', 'LIMIT', 'LOAN_BALANCE', 'INT_RATE'
    ]

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for row_data in cleaned_data:
            writer.writerow(row_data)

    print(f"Successfully cleaned data and saved to {output_csv_path}")
    print(f"Number of records extracted: {len(cleaned_data)}")

# --- Main execution ---
# In Colab, upload your file first, then set the path.
# For example, if your file is named "06083_20250228_Monthly_Report_on_Loans_Opened_Closed_CGL_Product_Wise_broc2525.txt"
file_path = '/content/06083_20250331_Monthly_Report_on_Loans_Opened_Closed_CGL_Product_Wise_broc2525.txt'
output_csv_path = 'OpenClose_Mar.csv'

try:
    with open(file_path, 'r') as f_test: # Test if file exists
        pass
    clean_monthly_report_to_csv(file_path, output_csv_path)
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please make sure you have uploaded the file to your Colab session and the path is correct.")

Successfully cleaned data and saved to OpenClose_Mar.csv
Number of records extracted: 333
